In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
import pandas as pd

In [3]:
# Import libraries
from llama_index.core import Document
import openai
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader # Correct
from llama_index.core import GPTVectorStoreIndex
import os
os.environ["GROQ_API_KEY"] = "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"
import os
from groq import Groq

In [54]:

client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Write a function to add a row in a CSV file. write in python",
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

**Adding a Row to a CSV File in Python**

Here's an example of a function that adds a row to a CSV file using the `csv` module in Python.

### Prerequisites

* Python 3.x
* `csv` module (part of the Python standard library)

### Code

```python
import csv

def add_row_to_csv(filename, row):
    """
    Add a row to a CSV file.

    Args:
        filename (str): The name of the CSV file.
        row (list): The row to add to the CSV file.

    Returns:
        None
    """
    with open(filename, 'a', newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(row)

# Example usage:
filename = 'example.csv'
row = ['John', 'Doe', '25']

# Create a new CSV file with a header row
with open(filename, 'w', newline='') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerow(['First Name', 'Last Name', 'Age'])

# Add a row to the CSV file
add_row_to_csv(filename, row)

# Add another row to the CSV file
add_row_to_csv(filename, ['Jane', 'Doe', '30'])
```

### E

In [88]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

In [35]:
# Install necessary packages if not already installed
# pip install llama-index groq

import os
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)
from groq import Groq

# Initialize Groq client
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

# Define custom events
class CodeStartEvent(StartEvent):
    request: str

class CodeGenerateEvent(Event):
    code: str

class CodeTestEvent(Event):
    result: str

# Define the workflow class
class CodeAssistantFlow(Workflow):
    @step
    
    # async def generate_code(self, ev: StartEvent) -> CodeGenerateEvent:
    #     prompt = f"Write Python code to {ev.request}."
    #     response = await self.llm.acomplete(prompt)
    #     code = str(response)
    #     return CodeGenerateEvent(code=code)
    
    
    

    async def generate_code(self, ev: StartEvent) -> CodeGenerateEvent:
        # Use Groq API to generate code
        # chat_completion = client.chat.completions.create(
        #     messages=[
        #         {
        #             "role": "user",
        #             "content": f"Write Python code to {ev.request}.",
        #         }
        #     ],
        #     model="llama-3-70b-versatile",  # Use the appropriate Groq model
        # )
        
        chat_completion = client.chat.completions.create(
            model="mixtral-8x7b-32768",
            messages=[
                {
                    "role": "system",
                    "content": "You are an AI-powered Python code generator. Your task is to generate clean, efficient, and well-commented Python code based on the user's request. Follow these guidelines:\n\nUnderstand the Task:\n\nCarefully interpret the user's programming request.\n\nIf the request is unclear, ask for clarification or make reasonable assumptions.\n\nGenerate Code:\n\nProvide Python code that solves the user's problem.\n\nInclude comments to explain key steps or logic.\n\nUse meaningful variable and function names for clarity.\n\nHandle potential edge cases and exceptions.\n\nCode Structure:\n\nOrganize the code logically, with functions or classes where appropriate.\n\nPrioritize readability and simplicity over unnecessary complexity.\n\nTesting Readiness:\n\nEnsure the code is ready to be tested or executed.\n\nIf the code requires user input, clearly indicate where and how the input should be provided.\n\nAssumptions:\n\nAssume all necessary libraries (e.g., pandas, numpy) are installed.\n\nIf the user does not specify details (e.g., file paths, data formats), make reasonable assumptions and state them clearly.\n\nError Handling:\n\nInclude basic error handling (e.g., try-except blocks) where applicable.\n\nInform the user of potential issues and how to resolve them.\n\nExample Output:\n\nProvide an example of how the code would work with sample data or inputs.\n\n"
        
                },
                {
                    "role": "user",
                    "content": f"Write Python code to {ev.request}.",
                },
            ],
            temperature=1,
            max_tokens=1024,
            top_p=1,
            stream=True,
            stop=None,
        )
        print(chat_completion)
        # k=""
        # for chunk in chat_completion:
        #     print(chunk.choices[0].delta.content or "", end="")
            
            
        j = ""
        # store the above printed message in a variable
        for chunk in chat_completion:
            k=chunk.choices[0].delta.content or ""
            # print(k)  
            j = j+k  
            
        #print the data type of k
        # print(type(k))
        
        print(j)
        
        
        # code = chat_completion.choices[0].message.content
        code = j
                
        # print(k)
        return CodeGenerateEvent(code=code)

    @step
    async def test_code(self, ev: CodeGenerateEvent) -> CodeTestEvent:
        # Simulate testing the generated code
        # For safety, return a placeholder result
        return CodeTestEvent(result="Code tested successfully.")

    @step
    async def provide_feedback(self, ev: CodeTestEvent) -> StopEvent:
        feedback = f"Test result: {ev.result}"
        return StopEvent(result=feedback)

# Run the workflow
if __name__ == "__main__":
    import asyncio

    w = CodeAssistantFlow(timeout=60, verbose=True)
    # result = asyncio.run(w.run(event=CodeStartEvent(request="Find the user with the highest total transaction amount from the CSV.")))
    # result = asyncio.run(w.run(request="Find the user with the highest total transaction amount from the CSV."))
    result = await w.run(request="Find the user with the highest total transaction amount from the CSV.")
    print(str(result))


Running step generate_code
Here's the Python code to find the user with the highest total transaction amount from a CSV file containing transaction data:

```python
import csv

def read_transaction_data(file_path):
    """
    Reads transaction data from a CSV file and returns a list of dictionaries.
    
    Each dictionary contains the transaction data for a single user, with
    keys 'user_id' and 'transaction_amount'.
    
Step generate_code produced event CodeGenerateEvent
Running step test_code
Step test_code produced event CodeTestEvent
Running step provide_feedback
Step provide_feedback produced event StopEvent
Test result: Code tested successfully.


In [54]:
system_prompt = ""

In [56]:
# import the system prompt from the file system-prompt.txt into the variable system_prompt
with open("system-prompt.txt", "r") as file:
    system_prompt = file.read()

In [57]:
system_prompt

"You are an AI-powered Python code generator. Your job is to generate clean, efficient, and well-commented Python code based on the user's request. Follow these guidelines:  \n\n### Task Understanding:  \n- Carefully read and interpret the user's programming request.  \n- If the request is unclear, ask for clarification or make reasonable assumptions.  \n\n### Code Generation:  \n- Write Python code that solves the user's problem directly, without adding unnecessary introductory lines.  \n- Ensure the code is provided as a `.py` file or a `.ipynb` notebook, depending on the task or user preference.  \n- Use concise, one-liner comments to explain key steps or logic in the code.  \n- Select meaningful variable and function names for clarity.  \n- Account for potential edge cases and include exception handling where applicable.  \n\n### Code Structure:  \n- Organize the code logically, using functions or classes as appropriate.  \n- Focus on simplicity and readability, avoiding unnecessar

In [79]:
from llama_index.core.workflow import (
    Event,
    StartEvent,
    StopEvent,
    Workflow,
    step,
)
from groq import Groq

# Initialize Groq client
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)

# Define custom events
class CodeStartEvent(StartEvent):
    request: str

class CodeGenerateEvent(Event):
    code: str

class CodeTestEvent(Event):
    result: str
    
# class CodeExplanationEvent(Event):
#     code: str  # The generated code
#     explanation: str  # Step-by-step explanation of the code    

# Define the workflow class
class CodeAssistantFlow(Workflow):
    @step
    async def generate_code(self, ev: StartEvent) -> CodeGenerateEvent:
        # Use Groq API to generate code
        chat_completion = client.chat.completions.create(
            # model="mixtral-8x7b-32768",
            model="llama-3.3-70b-versatile",
            messages=[
                {
                    "role": "system",
                    "content": f"{system_prompt}"
                },
                {
                    "role": "user",
                    "content": f"Write Python code to {ev.request}.",
                },
            ],
            temperature=1,  
            max_tokens=1024,
            top_p=1,
            stream=True,
            stop=None,
        )
        
        # response_accumulator is the varianble to store the response.   
        response_accumulator = ""
        # store the above printed message in a variable
        for chunk in chat_completion:
            k=chunk.choices[0].delta.content or ""
            # print(k)  
            response_accumulator += k  
        
        print(response_accumulator)
        code = response_accumulator
        return CodeGenerateEvent(code=code)

    @step
    async def test_code(self, ev: CodeGenerateEvent) -> CodeTestEvent:
        # Simulate testing the generated code
        # For safety, return a placeholder result
        return CodeTestEvent(result="Code tested successfully.")

    @step
    async def provide_feedback(self, ev: CodeTestEvent) -> StopEvent:
        feedback = f"Test result: {ev.result}"
        return StopEvent(result=feedback)
    
    # Run the workflow
if __name__ == "__main__":
    # Take request from the user
    user_request = input("Enter your coding task: ").strip()
    
    # Ensure the request is not empty
    if not user_request:
        print("No task provided. Exiting.")
    else:
        w = CodeAssistantFlow(timeout=60, verbose=True)
        result = await w.run(request=user_request)
        print(str(result))


Running step generate_code
```python
def add_numbers(a, b):
    # Attempt to add two numbers and handle potential type errors
    try:
        return a + b
    except TypeError:
        # Notify the user if inputs are not numbers
        print("Error: Both inputs must be numbers.")
        return None

# Example usage
num1 = 5
num2 = 7
result = add_numbers(num1, num2)

if result is not None:
    print(f"{num1} + {num2} = {result}")
```
Step generate_code produced event CodeGenerateEvent
Running step test_code
Step test_code produced event CodeTestEvent
Running step provide_feedback
Step provide_feedback produced event StopEvent
Test result: Code tested successfully.


In [85]:
from llama_index.core.workflow import draw_all_possible_flows
from llama_index.utils.workflow import draw_most_recent_execution

In [87]:
draw_all_possible_flows(CodeAssistantFlow, filename="trivial_workflow.html")

trivial_workflow.html


C:\Users\Admin\AppData\Local\Temp\ipykernel_14976\1684240689.py:1: DeprecationWarning: Call to deprecated function (or staticmethod) draw_all_possible_flows. (Install `llama-index-utils-workflow` and use the import `from llama_index.utils.workflow` instead.)
  draw_all_possible_flows(CodeAssistantFlow, filename="trivial_workflow.html")


In [61]:
# Run the workflow
if __name__ == "__main__":
    import asyncio

    w = CodeAssistantFlow(timeout=60, verbose=True)
    # result = asyncio.run(w.run(event=CodeStartEvent(request="Find the user with the highest total transaction amount from the CSV.")))
    # result = asyncio.run(w.run(request="Find the user with the highest total transaction amount from the CSV."))
    result = await w.run(request="Find the user with the highest total transaction amount from the CSV.")
    print(str(result))

Running step generate_code
import csv

def read_csv_file(file_path):
    """
    Reads a CSV file and returns a list of dictionaries, where each dictionary represents a row in the CSV.

    Args:
    file_path (str): The path to the CSV file.

    Returns:
    list: A list of dictionaries, where each dictionary contains the values of a row in the CSV.
    """
    transactions = []
    with open(file_path, newline='') as csvfile:
        reader = csv.DictReader(csvfile)
        for row in reader:
            transactions.append(row)
    return transactions

def get_user_with_highest_total_transaction(transactions):
    """
    Finds the user with the highest total transaction amount.

    Args:
    transactions (list): A list of dictionaries, where each dictionary represents a transaction and contains the user id and transaction amount.

    Returns:
    dict: A dictionary containing the user id and the highest total transaction amount.
    """
    user_transactions = {}
    highest_tot

In [62]:
# Run the workflow
if __name__ == "__main__":
    # Take request from the user
    user_request = input("Enter your coding task: ").strip()
    
    # Ensure the request is not empty
    if not user_request:
        print("No task provided. Exiting.")
    else:
        w = CodeAssistantFlow(timeout=60, verbose=True)
        result = await w.run(request=user_request)
        print(str(result))

Running step generate_code
Sure, here is a Python function that uses the `openai` library to call the OpenAI API. This function takes a prompt as input and generates a completion using the API. Make sure to install the `openai` library before running this code.

```python
# Import the necessary library
import openai

# Initialize the OpenAI API client with your API key
openai.api_key = "your_openai_api_key_here"

def call_openai_api(prompt: str) -> str:
    """
    Call the OpenAI API with a given prompt and return the generated completion.

    Parameters:
    prompt (str): The prompt to generate a completion for.

    Returns:
    str: The generated completion.

    Raises:
    Exception: If there is an error while calling the OpenAI API.
    """
    try:
        # Call the OpenAI API and get the generated completion
        completion = openai.Completion.create(
            engine="text-davinci-002",
            prompt=prompt,
            max_tokens=1024,
            n=1,
          